In [ ]:
import numpy as np
import pandas as pd

# 데이터 시각화 패키지
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib.sankey import Sankey  
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Jupyter Notebook에서 그래프를 바로 확인할 수 있도록 설정
%matplotlib inline

# 경고 메시지 무시
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (운영체제별 한글 지원)
from matplotlib import font_manager, rc

# '-' 기호가 제대로 보이도록 설정
matplotlib.rcParams['axes.unicode_minus'] = False

# 윈도우 환경: Malgun Gothic 폰트 설정
path = "c:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font', family=font_name)

In [3]:
pd.set_option('display.max_columns',None)

In [92]:
df1 = pd.read_csv('campaign_desc.csv')
df4 = pd.read_csv('coupon.csv')

In [ ]:
df1['dur'] = df1['end(day)'] - df1['start(day)']
df1 = df1.drop('Manager',axis=1)

In [8]:
df2 = pd.read_csv('campaign_table.csv')
df3 = pd.read_csv('coupon_used.csv')
df4 = pd.read_csv('coupon.csv')
df5 = pd.read_csv('customer_info.csv')
df6 = pd.read_csv('product.csv')
df8 = pd.read_csv('transaction_data.csv')

In [10]:
df1 = df1.drop('Campaign_Type',axis=1)

In [12]:
df21 = pd.merge(df2,df1, on='Campaign_ID', how='inner')
df21

Household_ID  Campaign_ID Campaign_Type  start(day)  end(day)  dur
0             1032           22             B         624       656   32
1              638           22             B         624       656   32
2              358           22             B         624       656   32
3             1376           22             B         624       656   32
4             1632           22             B         624       656   32
...            ...          ...           ...         ...       ...  ...
7203           304           27             C         237       300   63
7204          1917           27             C         237       300   63
7205          2185           27             C         237       300   63
7206          2317           27             C         237       300   63
7207          2200           27             C         237       300   63

[7208 rows x 6 columns]

In [19]:
df86 = pd.merge(df8, df6,on='Product_ID',how='inner')


In [ ]:

# 1. Household_ID와 Day 기준으로 거래 데이터와 쿠폰 사용 데이터를 merge (left join)
merged_df = pd.merge(df86, df3, 
                     how="left", 
                     on=["Household_ID", "Day"])

2648695

In [67]:
df3.groupby(["Household_ID", "Day"]).size().sort_values(ascending=False).head()

Household_ID  Day
931           602    19
2007          636    14
1197          522    13
2489          515    13
2400          600    11
dtype: int64

In [49]:

# 2. 거래 내역에 쿠폰이 적용되는지 여부 확인하기 위해, coupon_allowed_df와 coupon_code, Product_ID 기준으로 merge
df8634 = pd.merge(
    merged_df,
    df4,
    how="left",
    left_on=["Coupon_Code", "Product_ID"],
    right_on=["Coupon_Code", "Product_ID"]
)
# 여기서 merge 후에 coupon_allowed_df의 product_id(또는 product_id 컬럼)가 NaN이면, 해당 거래의 상품은 쿠폰 적용 대상이 아님


#df8634["coupon_code_final"] = df8634["Coupon_Code"].where(df4["Product_ID"].notnull(), None)
len(df8634)
# 4. 필요에 따라 merge 시 추가된 product_id 컬럼 제거 (깔끔하게)
#merged_final = merged_final.drop(columns=["product_id"])


2650935

In [50]:
df8634.drop('Campaign_ID_y',axis=1, inplace = True)

In [51]:
df8634.rename(columns={'Campaign_ID_x': 'Campaign_ID'}, inplace = True)
df8634.head()

Household_ID    Basket_ID  Product_ID  Store_ID  Day  Quantity  \
0          1803  30780785930     1065887       338  252         1   
1          1388  30253393814     1065887       375  228         1   
2          1268  32269557669     1065887     31742  362         1   
3           868  33944722714     1065887       323  467         1   
4          2374  40727257930     1065887       367  602         1   

   Sales_Value  Trans_time  Week_no  Disc(retail)  Disc(coupon)  \
0         8.99        1419       37          -1.0           0.0   
1         9.99        1748       33           0.0           0.0   
2         9.99        2014       52           0.0           0.0   
3         7.99        1829       67          -2.0           0.0   
4         9.99        1350       87           0.0           0.0   

   Disc(coupon_match)  Manufacturer    Brand Category Subcategory  \
0                 0.0            69  Private  DRUG GM  ANALGESICS   
1                 0.0            69  Private  DRUG GM  ANALGESICS   
2                 0.0            69  Private  DRUG GM  ANALGESICS   
3                 0.0            69  Private  DRUG GM  ANALGESICS   
4                 0.0            69  Private  DRUG GM  ANALGESICS   

       Product_type Curr_Size_of_Product  Coupon_Code  Campaign_ID  
0  ADULT ANALGESICS                               NaN          NaN  
1  ADULT ANALGESICS                               NaN          NaN  
2  ADULT ANALGESICS                               NaN          NaN  
3  ADULT ANALGESICS                               NaN          NaN  
4  ADULT ANALGESICS                               NaN          NaN

In [42]:
len(df8634)

2650935

In [52]:
coupon_dict = (
    df4
    .groupby('Coupon_Code')['Product_ID']
    .apply(set)
    .to_dict()
)


In [53]:

# 2. 행별로 쿠폰이 해당 제품에 적용되는지 판별하는 함수
def check_coupon(row):
    code = row['Coupon_Code']
    pid = row['Product_ID']
    if pd.isna(code):
        return pd.NA
    if code in coupon_dict and pid in coupon_dict[code]:
        return code
    return pd.NA

# 3. 적용
df8634['coupon_code_final'] = df8634.apply(check_coupon, axis=1)

In [73]:
df8634[(df8634['Household_ID'] == 2489) & (df8634['Day'] == 515)].sort_values(by='Product_ID').head(15)

Household_ID    Basket_ID  Product_ID  Store_ID  Day  Quantity  \
366188           2489  35145556678      822241       362  515         4   
366189           2489  35145556678      822241       362  515         4   
366187           2489  35145556678      822241       362  515         4   
366186           2489  35145556678      822241       362  515         4   
366185           2489  35145556678      822241       362  515         4   
366183           2489  35145556678      822241       362  515         4   
366184           2489  35145556678      822241       362  515         4   
366181           2489  35145556678      822241       362  515         4   
366180           2489  35145556678      822241       362  515         4   
366179           2489  35145556678      822241       362  515         4   
366178           2489  35145556678      822241       362  515         4   
366177           2489  35145556678      822241       362  515         4   
366182           2489  35145556678      822241       362  515         4   
2068692          2489  35145556678      826571       362  515         1   
2068689          2489  35145556678      826571       362  515         1   

         Sales_Value  Trans_time  Week_no  Disc(retail)  Disc(coupon)  \
366188          0.80        1819       74          0.00          0.00   
366189          0.80        1819       74          0.00          0.00   
366187          0.80        1819       74          0.00          0.00   
366186          0.80        1819       74          0.00          0.00   
366185          0.80        1819       74          0.00          0.00   
366183          0.80        1819       74          0.00          0.00   
366184          0.80        1819       74          0.00          0.00   
366181          0.80        1819       74          0.00          0.00   
366180          0.80        1819       74          0.00          0.00   
366179          0.80        1819       74          0.00          0.00   
366178          0.80        1819       74          0.00          0.00   
366177          0.80        1819       74          0.00          0.00   
366182          0.80        1819       74          0.00          0.00   
2068692         2.05        1819       74         -0.49         -0.55   
2068689         2.05        1819       74         -0.49         -0.55   

         Disc(coupon_match)  Manufacturer     Brand Category  \
366188                 0.00          1046  National  GROCERY   
366189                 0.00          1046  National  GROCERY   
366187                 0.00          1046  National  GROCERY   
366186                 0.00          1046  National  GROCERY   
366185                 0.00          1046  National  GROCERY   
366183                 0.00          1046  National  GROCERY   
366184                 0.00          1046  National  GROCERY   
366181                 0.00          1046  National  GROCERY   
366180                 0.00          1046  National  GROCERY   
366179                 0.00          1046  National  GROCERY   
366178                 0.00          1046  National  GROCERY   
366177                 0.00          1046  National  GROCERY   
366182                 0.00          1046  National  GROCERY   
2068692               -0.45           857  National  GROCERY   
2068689               -0.45           857  National  GROCERY   

                            Subcategory               Product_type  \
366188              PWDR/CRYSTL DRNK MX  SOFT DRINK POWDER POUCHES   
366189              PWDR/CRYSTL DRNK MX  SOFT DRINK POWDER POUCHES   
366187              PWDR/CRYSTL DRNK MX  SOFT DRINK POWDER POUCHES   
366186              PWDR/CRYSTL DRNK MX  SOFT DRINK POWDER POUCHES   
366185              PWDR/CRYSTL DRNK MX  SOFT DRINK POWDER POUCHES   
366183              PWDR/CRYSTL DRNK MX  SOFT DRINK POWDER POUCHES   
366184              PWDR/CRYSTL DRNK MX  SOFT DRINK POWDER POUCHES   
366181              PWDR/CRYSTL DRNK MX  SOFT DRINK PO

In [74]:
coupon_per_basket = (
    df8634[~df8634['coupon_code_final'].isna()]
    .groupby('Basket_ID')['coupon_code_final']
    .agg(lambda x: sorted(x.unique())[0])  # 여러 쿠폰이면 정렬 후 첫 번째로 통일
    .reset_index()
)

In [75]:
df = pd.merge(df8634, coupon_per_basket, on="Basket_ID", how="left", suffixes=('', '_basket'))
df['coupon_code_final_fixed'] = df['coupon_code_final'].combine_first(df['coupon_code_final_basket'])

In [77]:
df_compare = df.dropna(subset=["Coupon_Code", "coupon_code_final_fixed"])

# 2. 값이 다른 행만 추출
wrong_rows = df_compare[df_compare["Coupon_Code"] != df_compare["coupon_code_final_fixed"]]

# 3. 전체에서 제거
df_cleaned = df.drop(index=wrong_rows.index).reset_index(drop=True)

In [78]:
df_cleaned

Household_ID    Basket_ID  Product_ID  Store_ID  Day  Quantity  \
0                1803  30780785930     1065887       338  252         1   
1                1388  30253393814     1065887       375  228         1   
2                1268  32269557669     1065887     31742  362         1   
3                 868  33944722714     1065887       323  467         1   
4                2374  40727257930     1065887       367  602         1   
...               ...          ...         ...       ...  ...       ...   
2601489          2224  30579241929      928870       315  236         1   
2601490          2184  28344932057     9803768       309  114         1   
2601491            57  31344122693     6039807       298  293         1   
2601492          1835  27469178389     1037872       375   41         1   
2601493          1409  28856455232     9423802      3313  145         1   

         Sales_Value  Trans_time  Week_no  Disc(retail)  Disc(coupon)  \
0               8.99        1419       37         -1.00           0.0   
1               9.99        1748       33          0.00           0.0   
2               9.99        2014       52          0.00           0.0   
3               7.99        1829       67         -2.00           0.0   
4               9.99        1350       87          0.00           0.0   
...              ...         ...      ...           ...           ...   
2601489         4.50        1813       34          0.00           0.0   
2601490         7.99        2119       17          0.00           0.0   
2601491         6.04        1845       43          0.00           0.0   
2601492         6.99        1830        7          0.00           0.0   
2601493         1.67         929       21         -0.32           0.0   

         Disc(coupon_match)  Manufacturer     Brand   Category  \
0                       0.0            69   Private    DRUG GM   
1                       0.0            69   Private    DRUG GM   
2                       0.0            69   Private    DRUG GM   
3                       0.0            69   Private    DRUG GM   
4                       0.0            69   Private    DRUG GM   
...                     ...           ...       ...        ...   
2601489                 0.0           504  National    DRUG GM   
2601490                 0.0          5014  National  COSMETICS   
2601491                 0.0          5016  National  COSMETICS   
2601492                 0.0          2239  National    DRUG GM   
2601493                 0.0          6351  National    GROCERY   

                  Subcategory                Product_type  \
0                  ANALGESICS            ADULT ANALGESICS   
1                  ANALGESICS            ADULT ANALGESICS   
2                  ANALGESICS            ADULT ANALGESICS   
3                  ANALGESICS            ADULT ANALGESICS   
4                  ANALGESICS            ADULT ANALGESICS   
...                       ...                         ...   
2601489              MAGAZINE         AUTOMOBILE-MAGAZINE   
2601490  MAKEUP AND TREATMENT  FACE MAKE UP AND TREATMENT   
2601491  MAKEUP AND TREATMENT                      REVLON   
2601492                SUNTAN                      SUNTAN   
2601493     DRY NOODLES/PASTA               SPAGHETTI DRY   

        Curr_Size_of_Product  Coupon_Code  Campaign_ID coupon_code_final  \
0                                     NaN          NaN              <NA>   
1                                     NaN          NaN              <NA>   
2                                     NaN          NaN              <NA>   
3                                     NaN          NaN              <NA>   
4                                     NaN          NaN              <NA>   
...                      ...          ...          ...               ...   
2601489                               NaN          NaN              <NA>   
2601490                               NaN          NaN              <NA>   
2601491              

In [80]:
df_real = df_cleaned.drop(['coupon_code_final_basket','coupon_code_final_fixed'], axis=1)

In [86]:
df_real = df_real.drop_duplicates()

In [87]:
df_real

Household_ID    Basket_ID  Product_ID  Store_ID  Day  Quantity  \
0                1803  30780785930     1065887       338  252         1   
1                1388  30253393814     1065887       375  228         1   
2                1268  32269557669     1065887     31742  362         1   
3                 868  33944722714     1065887       323  467         1   
4                2374  40727257930     1065887       367  602         1   
...               ...          ...         ...       ...  ...       ...   
2601489          2224  30579241929      928870       315  236         1   
2601490          2184  28344932057     9803768       309  114         1   
2601491            57  31344122693     6039807       298  293         1   
2601492          1835  27469178389     1037872       375   41         1   
2601493          1409  28856455232     9423802      3313  145         1   

         Sales_Value  Trans_time  Week_no  Disc(retail)  Disc(coupon)  \
0               8.99        1419       37         -1.00           0.0   
1               9.99        1748       33          0.00           0.0   
2               9.99        2014       52          0.00           0.0   
3               7.99        1829       67         -2.00           0.0   
4               9.99        1350       87          0.00           0.0   
...              ...         ...      ...           ...           ...   
2601489         4.50        1813       34          0.00           0.0   
2601490         7.99        2119       17          0.00           0.0   
2601491         6.04        1845       43          0.00           0.0   
2601492         6.99        1830        7          0.00           0.0   
2601493         1.67         929       21         -0.32           0.0   

         Disc(coupon_match)  Manufacturer     Brand   Category  \
0                       0.0            69   Private    DRUG GM   
1                       0.0            69   Private    DRUG GM   
2                       0.0            69   Private    DRUG GM   
3                       0.0            69   Private    DRUG GM   
4                       0.0            69   Private    DRUG GM   
...                     ...           ...       ...        ...   
2601489                 0.0           504  National    DRUG GM   
2601490                 0.0          5014  National  COSMETICS   
2601491                 0.0          5016  National  COSMETICS   
2601492                 0.0          2239  National    DRUG GM   
2601493                 0.0          6351  National    GROCERY   

                  Subcategory                Product_type  \
0                  ANALGESICS            ADULT ANALGESICS   
1                  ANALGESICS            ADULT ANALGESICS   
2                  ANALGESICS            ADULT ANALGESICS   
3                  ANALGESICS            ADULT ANALGESICS   
4                  ANALGESICS            ADULT ANALGESICS   
...                       ...                         ...   
2601489              MAGAZINE         AUTOMOBILE-MAGAZINE   
2601490  MAKEUP AND TREATMENT  FACE MAKE UP AND TREATMENT   
2601491  MAKEUP AND TREATMENT                      REVLON   
2601492                SUNTAN                      SUNTAN   
2601493     DRY NOODLES/PASTA               SPAGHETTI DRY   

        Curr_Size_of_Product  Coupon_Code  Campaign_ID coupon_code_final  
0                                     NaN          NaN              <NA>  
1                                     NaN          NaN              <NA>  
2                                     NaN          NaN              <NA>  
3                                     NaN          NaN              <NA>  
4                                     NaN          NaN              <NA>  
...                      ...          ...          ...               ...  
2601489                               NaN          NaN              <NA>  
2601490                               NaN          NaN              <NA>  
2601491                       

In [97]:
df_rreal = pd.merge(
    df_real,
    df1[["Campaign_ID", "Campaign_Type", "start(day)", "end(day)", "dur"]],
    how="left",
    on="Campaign_ID"
)
df_rreal

Household_ID    Basket_ID  Product_ID  Store_ID  Day  Quantity  \
0                1803  30780785930     1065887       338  252         1   
1                1388  30253393814     1065887       375  228         1   
2                1268  32269557669     1065887     31742  362         1   
3                 868  33944722714     1065887       323  467         1   
4                2374  40727257930     1065887       367  602         1   
...               ...          ...         ...       ...  ...       ...   
2599249          2224  30579241929      928870       315  236         1   
2599250          2184  28344932057     9803768       309  114         1   
2599251            57  31344122693     6039807       298  293         1   
2599252          1835  27469178389     1037872       375   41         1   
2599253          1409  28856455232     9423802      3313  145         1   

         Sales_Value  Trans_time  Week_no  Disc(retail)  Disc(coupon)  \
0               8.99        1419       37         -1.00           0.0   
1               9.99        1748       33          0.00           0.0   
2               9.99        2014       52          0.00           0.0   
3               7.99        1829       67         -2.00           0.0   
4               9.99        1350       87          0.00           0.0   
...              ...         ...      ...           ...           ...   
2599249         4.50        1813       34          0.00           0.0   
2599250         7.99        2119       17          0.00           0.0   
2599251         6.04        1845       43          0.00           0.0   
2599252         6.99        1830        7          0.00           0.0   
2599253         1.67         929       21         -0.32           0.0   

         Disc(coupon_match)  Manufacturer     Brand   Category  \
0                       0.0            69   Private    DRUG GM   
1                       0.0            69   Private    DRUG GM   
2                       0.0            69   Private    DRUG GM   
3                       0.0            69   Private    DRUG GM   
4                       0.0            69   Private    DRUG GM   
...                     ...           ...       ...        ...   
2599249                 0.0           504  National    DRUG GM   
2599250                 0.0          5014  National  COSMETICS   
2599251                 0.0          5016  National  COSMETICS   
2599252                 0.0          2239  National    DRUG GM   
2599253                 0.0          6351  National    GROCERY   

                  Subcategory                Product_type  \
0                  ANALGESICS            ADULT ANALGESICS   
1                  ANALGESICS            ADULT ANALGESICS   
2                  ANALGESICS            ADULT ANALGESICS   
3                  ANALGESICS            ADULT ANALGESICS   
4                  ANALGESICS            ADULT ANALGESICS   
...                       ...                         ...   
2599249              MAGAZINE         AUTOMOBILE-MAGAZINE   
2599250  MAKEUP AND TREATMENT  FACE MAKE UP AND TREATMENT   
2599251  MAKEUP AND TREATMENT                      REVLON   
2599252                SUNTAN                      SUNTAN   
2599253     DRY NOODLES/PASTA               SPAGHETTI DRY   

        Curr_Size_of_Product  Coupon_Code  Campaign_ID coupon_code_final  \
0                                     NaN          NaN               NaN   
1                                     NaN          NaN               NaN   
2                                     NaN          NaN               NaN   
3                                     NaN          NaN               NaN   
4                                     NaN          NaN               NaN   
...                      ...          ...          ...               ...   
2599249                               NaN          NaN               NaN   
2599250                               NaN          NaN               NaN   
2599251              

In [147]:
dup_mask = df_rreal.duplicated(subset=["Household_ID", "Day", "Product_ID", "Basket_ID"], keep=False)

# 2. 중복된 모든 행들의 특정 컬럼값 NaN으로 설정
df_rreal.loc[dup_mask, "Coupon_Code"] = pd.NA

# 3. 중복 제거 (첫 번째만 남기기)
df_cleaned2 = df_rreal.drop_duplicates(subset=["Household_ID", "Day", "Product_ID", "Basket_ID"], keep="first").reset_index(drop=True)

In [149]:
df_cleaned2

Household_ID    Basket_ID  Product_ID  Store_ID  Day  Quantity  \
0                1803  30780785930     1065887       338  252         1   
1                1388  30253393814     1065887       375  228         1   
2                1268  32269557669     1065887     31742  362         1   
3                 868  33944722714     1065887       323  467         1   
4                2374  40727257930     1065887       367  602         1   
...               ...          ...         ...       ...  ...       ...   
2595727          2224  30579241929      928870       315  236         1   
2595728          2184  28344932057     9803768       309  114         1   
2595729            57  31344122693     6039807       298  293         1   
2595730          1835  27469178389     1037872       375   41         1   
2595731          1409  28856455232     9423802      3313  145         1   

         Sales_Value  Trans_time  Week_no  Disc(retail)  Disc(coupon)  \
0               8.99        1419       37         -1.00           0.0   
1               9.99        1748       33          0.00           0.0   
2               9.99        2014       52          0.00           0.0   
3               7.99        1829       67         -2.00           0.0   
4               9.99        1350       87          0.00           0.0   
...              ...         ...      ...           ...           ...   
2595727         4.50        1813       34          0.00           0.0   
2595728         7.99        2119       17          0.00           0.0   
2595729         6.04        1845       43          0.00           0.0   
2595730         6.99        1830        7          0.00           0.0   
2595731         1.67         929       21         -0.32           0.0   

         Disc(coupon_match)  Manufacturer     Brand   Category  \
0                       0.0            69   Private    DRUG GM   
1                       0.0            69   Private    DRUG GM   
2                       0.0            69   Private    DRUG GM   
3                       0.0            69   Private    DRUG GM   
4                       0.0            69   Private    DRUG GM   
...                     ...           ...       ...        ...   
2595727                 0.0           504  National    DRUG GM   
2595728                 0.0          5014  National  COSMETICS   
2595729                 0.0          5016  National  COSMETICS   
2595730                 0.0          2239  National    DRUG GM   
2595731                 0.0          6351  National    GROCERY   

                  Subcategory                Product_type  \
0                  ANALGESICS            ADULT ANALGESICS   
1                  ANALGESICS            ADULT ANALGESICS   
2                  ANALGESICS            ADULT ANALGESICS   
3                  ANALGESICS            ADULT ANALGESICS   
4                  ANALGESICS            ADULT ANALGESICS   
...                       ...                         ...   
2595727              MAGAZINE         AUTOMOBILE-MAGAZINE   
2595728  MAKEUP AND TREATMENT  FACE MAKE UP AND TREATMENT   
2595729  MAKEUP AND TREATMENT                      REVLON   
2595730                SUNTAN                      SUNTAN   
2595731     DRY NOODLES/PASTA               SPAGHETTI DRY   

        Curr_Size_of_Product  Coupon_Code  Campaign_ID coupon_code_final  \
0                                     NaN          NaN              <NA>   
1                                     NaN          NaN              <NA>   
2                                     NaN          NaN              <NA>   
3                                     NaN          NaN              <NA>   
4                                     NaN          NaN              <NA>   
...                      ...          ...          ...               ...   
2595727                               NaN          NaN              <NA>   
2595728                               NaN          NaN              <NA>   
2595729              

In [150]:
df_cleaned2.to_csv('all_merged_except_panchok.csv',index=False)